## Multiple inputs

In [1]:
from keras.models import Model
from keras.layers import Embedding, LSTM, Dense, concatenate
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = LSTM(16)(embedded_question)

concatenated = concatenate([encoded_text, encoded_question], axis=-1)
answer = Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [3]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                            size = (num_samples, max_length))

answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
    x[indices[i]] = 1

In [4]:
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 5s 127ms/step - loss: 6.2149 - acc: 0.0010
Epoch 2/10
8/8 [==============================] - 1s 124ms/step - loss: 6.1973 - acc: 0.0410
Epoch 3/10
8/8 [==============================] - 1s 127ms/step - loss: 6.1440 - acc: 0.0150
Epoch 4/10
8/8 [==============================] - 1s 120ms/step - loss: 6.0539 - acc: 0.0080
Epoch 5/10
8/8 [==============================] - 1s 121ms/step - loss: 5.9858 - acc: 0.0120
Epoch 6/10
8/8 [==============================] - 1s 120ms/step - loss: 5.8827 - acc: 0.0080
Epoch 7/10
8/8 [==============================] - 1s 121ms/step - loss: 5.7778 - acc: 0.0090
Epoch 8/10
8/8 [==============================] - 1s 121ms/step - loss: 5.6917 - acc: 0.0310
Epoch 9/10
8/8 [==============================] - 1s 121ms/step - loss: 5.6198 - acc: 0.0440
Epoch 10/10
8/8 [==============================] - 1s 119ms/step - loss: 5.5485 - acc: 0.0480


## Multiple outputs

In [5]:
from keras.models import Model
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Embedding, Dense
from keras import Input

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = Embedding(256, vocabulary_size)(posts_input)
x = Conv1D(128, 5, activation='relu')(embedded_posts)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = Conv1D(256, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)

age_prediction = Dense(1, name='age')(x)
income_prediction = Dense(num_income_groups,
                          activation='softmax',
                          name='income')(x)
gender_prediction = Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])

In [7]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

## Module Inception

In [ ]:
from keras.layers import Conv2D, AveragePooling2D, concatenate

branch_a = Conv2D(128, 1,
                  activation='relu', strides=2)(x)
branch_b = Conv2D(128, 1, activation='relu')(x)
branch_b = Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = AveragePooling2D(3, strides=2)(x)
branch_c = Conv2D(128, 3, activation='relu')(branch_c)

branch_d = Conv2D(128, 1, activation='relu')
branch_d = Conv2D(128, 3, activation='relu')
branch_d = Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

## Vision model with common convolutional basis

In [9]:
from keras.layers import concatenate
from keras.applications.xception import Xception
from keras import Input

xception_base = Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = concatenate([left_features, right_features], axis=-1)